## API

L'exercice pour le cours prochain est le suivant: 
- Récupérer via crawling la liste des 256 top contributors sur cette page https://gist.github.com/paulmillr/2657075 
- En utilisant l'API github https://developer.github.com/v3/ récupérer pour chacun de ces users le nombre moyens de stars des repositories qui leur appartiennent. Pour finir classer ces 256 contributors par leur note moyenne.﻿

In [60]:
import threading
import time
import json
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [61]:
# Alimentation de la liste des 256 top users
def getTopGitHubbers() : 
    url = "https://gist.github.com/paulmillr/2657075"
    res = requests.get(url)
    list_users=[]
    if  res.status_code == 200:
        html_doc =  res.text
        soup = BeautifulSoup(html_doc,"html.parser")
        user_selection = soup.select('th + td a')
        for user in user_selection:
            list_users.append(user.text)
    return list_users

In [62]:
# Récupération du nombre moyen d'étoiles
def getUserStars(i,user) :
    try:
        token = 'e9211500391635b2beb9906d955b04d7e250cb19'
        url = 'https://api.github.com/users/' + user + '/repos?access_token=' + token + '&per_page=1000'
        time.sleep(0.8)
        # Non géré : si plus de 100 repository, la moyenne se fera sur les 100 premiers repositorys
        res = requests.get(url)
        if  res.status_code == 200:
            html_doc =  res.text
            soup = BeautifulSoup(html_doc,"html.parser")
            datas = json.loads(soup.text)
            stars_counter = []
            stargazersCountList = [item['stargazers_count'] for item in datas]
            for item in stargazersCountList:
                stars_counter.append(item)
            if len(stars_counter)>0:     
                meanS = np.mean(stars_counter)
            else : meanS = 0  
            verrou.acquire()
            dict_stars[user] = meanS
            verrou.release()
        else:
            print("pb de lecture api sur user : %s" % (user))
    except:
        print("une exception")

In [63]:
# Tache pour chaque thread : traiter une partie de la liste des users       
def multitask(i,list_u):
    print(("thread %d : %s " % (i,len(list_u))))
    for user in list_u:
        #print("thread %d : %s " % (i,user))
        getUserStars(1, user)

In [65]:
# Dictionnaire
dict_stars={}
# Verrou pour l'écriture de la ressource partagée de dictionnaire
verrou = threading.Lock()
# Chrono
start = time.time()     
print("Démarrage du traitement")
# Liste des 256 top users
list_users=[]
list_users=getTopGitHubbers()
n = 256
# Nbre de threads // à lancer
ndiv=4    
# liste de threads
t = []
# Lancement des threads 
for i in range(ndiv):
    #print("Thread %d : %d à %d" % (i,(i * (n//ndiv)),((i+1) * (n//ndiv))))
    thread = threading.Thread(target=multitask, args=(i, list_users[(i * (n//ndiv)):((i+1) * (n//ndiv))]))
    thread.start()
    t.append(thread)
        
# Attente de la fin d'exécution de l'ensemble des threads
for thread in t:
    thread.join()
# Tri
sorted_d = {}
print(len(dict_stars))
sorted_d = sorted(dict_stars.items(), key=lambda x: x[1], reverse=True)
# Fin chrono
end = time.time()
print("Durée du traitement %0.2f" % (end - start))
# Stockage et Affichage
df = pd.DataFrame.from_records(sorted_d,columns=['user','nb moyen étoiles'])
df

256
Durée du traitement 130.89


,user,nb moyen étoiles
0,bailicangdu,3503.538462
1,kamranahmedse,2275.272727
2,vinta,1830.343750
3,mrdoob,1597.228571
4,IanLunn,1402.055556
5,sebastianbergmann,1388.183333
6,junyanz,1373.071429
7,nolimits4web,1338.928571
8,ruanyf,1261.547170
9,JakeWharton,1072.980000
